### *"Courage is NOT having the strength to go on; it is going on when you don't have the strength."*
*-Theodore Roosevelt*

So be courageous!  The end of the semester is in sight!

----

YOUR NAME HERE: TJ Augustine

----

# HW 8: Correlated Queries, Group By/Having, and Join

In [123]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sqlite3 as sql



## Review

Based on what you have learned, here's the general form of a select statement.

```SELECT column_name(s)```

```FROM table_name```

```WHERE condition```

```GROUP BY column_name(s)```

```HAVING condition```

```ORDER BY column_name(s)```


Now remember:  

The ```HAVING``` clause is like a WHERE clause but is used when the selection condition is based on aggregate funtion of a column, such as ```COUNT(name) > 10``` or ```MAX(Rain) < 2.0```.

In this homework, we will review the select statement, incoporpating correlated subqueries and focusing on the GROUP BY and HAVING functionality as well as the JOIN functionality.  To make sure you are understanding when to use the capabilities and when to not, some problems will not require these techniques, so think hard before assuming you need one of these keywords.

----

Let's bring in the weather data.

Recall that the table contains about 62 years worth of temperature, rainfall and sunshine data for the city of London.

There is a record/row for each year/month from Jan 1957 to April 2019.  

* Tmax:  The max temp for the month (celsius)
* Tmin:  The min temp for the month (celsius)
* Rain: the rainfall is the total amout of rain for the month in millimeters
* Sun : the total number of hours sunshine for the month.

In [ ]:
#Read in a csv with weather data
weather = pd.read_csv('weather.csv')
print(weather)
conn = sql.connect('weather.db')
weather.to_sql('weather', conn)

### Don't forget to use your SQL Lab notes! 


----

### Question 1.

Except for 2019, there are 12 rows per year in the weather data beacuse we have monthly weather measurements.  

Find the average rainfall per year, except for 2019.

So your final answer will be as follows.

| Year | AVG(Rain) |
|:----:|:---------:|
| 1957 | 46.466667 |
| 1958 | 61.641667 |
| 1959 | 40.608333 |
| 1960 | 61.283333 |
| 1961 | 46.566667 |
|  ... |    ...    |
| 2014 | 72.000000 |
| 2015 | 46.833333 |
| 2016 | 49.216667 |
| 2017 | 47.683333 |
| 2018 | 48.333333 |

In [46]:
sql_statement = """SELECT year, AVG(Rain)
                   FROM weather
                   GROUP BY Year HAVING Year < 2019"""

weather_results = pd.read_sql_query(sql_statement, conn)
weather_results

,Year,AVG(Rain)
0,1957,46.466667
1,1958,61.641667
2,1959,40.608333
3,1960,61.283333
4,1961,46.566667
...,...,...
57,2014,72.000000
58,2015,46.833333
59,2016,49.216667
60,2017,47.683333


### Question 2.

Select all the rows where the Tmin and Tmax are within 4.5 degrees celsius of each other.  Show the Year, Month, Tmin, and Tmax.

HINTS:  
* The result should be a 7 row X 4 column table.
* The first row should be Jan 1966 with a Tmin of 1.0 degree celsius and a Tmax of 5.3 degrees celsius

In [47]:
sql_statement = """SELECT Year, Month, Tmin, Tmax
                   FROM weather
                   WHERE (Tmax-Tmin <= 4.5)"""

weather_results = pd.read_sql_query(sql_statement, conn)
weather_results 

,Year,Month,Tmin,Tmax
0,1966,1,1.0,5.3
1,1968,12,1.0,5.4
2,1986,2,-2.7,1.7
3,1996,1,3.1,7.2
4,2000,12,4.6,9.0
5,2002,12,5.2,9.0
6,2013,1,2.0,6.5


### Question 3.

Like us, the summer months in London are June, July, and August.

Find the years where the average Tmax during the summer months is greater than the average overall summer Tmax.  Show the average/Max/Min Tmax for these years.

The answer will have 29 rows and is shown below.

| Year | AVG(Tmax) | MAX(Tmax) | MIN(Tmax) |
|:----:|:---------:|:---------:|:---------:|
| 1957 | 22.400000 |    23.6   |    21.1   |
| 1959 | 23.666667 |    24.7   |    22.1   |
| 1970 | 22.400000 |    23.6   |    21.3   |
| 1973 | 22.333333 |    23.7   |    21.4   |
| 1975 | 23.933333 |    25.9   |    21.8   |
| 1976 | 25.733333 |    26.6   |    25.1   |
| 1983 | 24.300000 |    27.6   |    20.8   |
| 1984 | 23.300000 |    24.4   |    21.3   |
| 1989 | 24.033333 |    25.8   |    22.1   |
| 1990 | 23.400000 |    26.0   |    19.5   |
|  ... |    ...    |    ...    |    ...    |
| 2010 | 23.366667 |    25.0   |    21.6   |
| 2013 | 23.866667 |    27.0   |    20.3   |
| 2014 | 23.200000 |    25.8   |    21.7   |
| 2015 | 22.700000 |    23.7   |    22.2   |
| 2016 | 23.133333 |    24.7   |    20.7   |
| 2017 | 23.266667 |    24.0   |    22.0   |
| 2018 | 25.666667 |    28.3   |    24.2   |

In [166]:
sql_statement = """SELECT Year, AVG(Tmax), MAX(Tmax), MIN(Tmax)
                   FROM weather 
                   WHERE Month in (6,7,8)
                   GROUP BY Year HAVING AVG(Tmax) > (SELECT AVG(Tmax)
                                                     FROM weather
                                                     WHERE Month in (6, 7, 8))
                   """
                   

#FIX THISSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSS
weather_results = pd.read_sql_query(sql_statement, conn)
weather_results

,Year,AVG(Tmax),MAX(Tmax),MIN(Tmax)
0,1957,22.400000,23.6,21.1
1,1959,23.666667,24.7,22.1
2,1970,22.400000,23.6,21.3
3,1973,22.333333,23.7,21.4
4,1975,23.933333,25.9,21.8
5,1976,25.733333,26.6,25.1
6,1983,24.300000,27.6,20.8
7,1984,23.300000,24.4,21.3
8,1989,24.033333,25.8,22.1
9,1990,23.400000,26.0,19.5


----
### Question 4.

*Without hardcoding*, select only the years and months that are within 3 degrees of the MIN temperature in the DB.  
 
 HINTS: 
* ```SElECT A-10 FROM....``` returns all the values in the column A reduced by 10.
*  The answer  is as follows.

| Year | Month |
|:----:|:-----:|
| 1963 |   1   |
| 1963 |   2   |
| 1979 |   1   |
| 1985 |   1   |
| 1986 |   2   |

In [21]:
sql_statement = """SELECT Year, Month
                   FROM weather
                   WHERE Tmin < (SELECT MIN(Tmin)
                                 FROM weather) +3
                   """
weather_results = pd.read_sql_query(sql_statement, conn)
weather_results

,Year,Month
0,1963,1
1,1963,2
2,1979,1
3,1985,1
4,1986,2


### Closing the DB connection.
Let's close the DB connection as we will now work with  a different data set.

In [ ]:
conn.close()

----

### NBA Salaries

First, import the NBA Salaries from the 2015-2016 season for a few more practice problems.

In [ ]:
nba_15_16 = pd.read_csv('nba_salaries.csv') #Open the csv
conn = sql.connect('nba_more.db') #Create an nba_more database
nba_15_16.to_sql('nba_15_16', conn)  #Dump the csv into a table called nba_15_16 in the nba_more database

Let's select everything form the table, so you can remember what is in this nba dataset.

In [126]:
sql_statement = """SELECT *
                   FROM nba_15_16"""

nba_results = pd.read_sql_query(sql_statement, conn)
nba_results

,index,player,position,team,salary
0,0,Paul Millsap,PF,Atlanta Hawks,18.671659
1,1,Al Horford,C,Atlanta Hawks,12.000000
2,2,Tiago Splitter,C,Atlanta Hawks,9.756250
3,3,Jeff Teague,PG,Atlanta Hawks,8.000000
4,4,Kyle Korver,SG,Atlanta Hawks,5.746479
...,...,...,...,...,...
412,412,Gary Neal,PG,Washington Wizards,2.139000
413,413,DeJuan Blair,C,Washington Wizards,2.000000
414,414,Kelly Oubre Jr.,SF,Washington Wizards,1.920240
415,415,Garrett Temple,SG,Washington Wizards,1.100602


----
### Question 5.

Without hardcoding, find the names and salaries of any players on The 76ers who make no less than Nik Stauskas.

The answer is:

|     player     |   salary  |
|:--------------:|:---------:|
| Gerald Wallace | 10.105855 |
|   Carl Landry  |  6.500000 |
|   Joel Embiid  |  4.626960 |
|  Jahlil Okafor |  4.582680 |
|  Nerlens Noel  |  3.457800 |
|  Nik Stauskas  |  2.869440 |

In [127]:
sql_statement = """SELECT player, salary
                   FROM nba_15_16
                   WHERE team = 'Philadelphia 76ers'
                   AND salary >= (SELECT salary
                                  FROM nba_15_16
                                  WHERE player = 'Nik Stauskas')"""

nba_results = pd.read_sql_query(sql_statement, conn)
nba_results


,player,salary
0,Gerald Wallace,10.105855
1,Carl Landry,6.500000
2,Joel Embiid,4.626960
3,Jahlil Okafor,4.582680
4,Nerlens Noel,3.457800
5,Nik Stauskas,2.869440


----
### Question 6.

This is the tough one!

Select all teams where ALL point guards ('PG' as the position) made more than 2 million dollars.  There should be no repeats on your final list.

Hints:  

* If you want a code hint, double click on the cell below.

* Otherwise, know that the answer should be as follows.

|          team          |
|:----------------------:|
|      Chicago Bulls     |
|    Dallas Mavericks    |
|     Denver Nuggets     |
|     Detroit Pistons    |
|  Golden State Warriors |
|     Houston Rockets    |
|   Los Angeles Lakers   |
|  Oklahoma City Thunder |
|   Philadelphia 76ers   |
| Portland Trail Blazers |
|    Sacramento Kings    |
|    Washington Wizard   |

Double-click __here__ for a code hint.

<!-- Start here.

sql_statement = """SELECT team
                   FROM nba AS x
                   WHERE 2.0 <= ??? """

-->
<hr/>

In [152]:
sql_statement = """SELECT DISTINCT(team)
                   FROM nba_15_16 x
                   WHERE 2.0 <= (SELECT MIN(salary)
                                 FROM nba_15_16 y
                                 WHERE x.team = y.team
                                 AND y.position = 'PG')
                   AND x.position = 'PG'
                   """
sql_statement2 = """SELECT team
                    FROM nba_15_16
                    WHERE position = 'PG'
                    GROUP BY team HAVING (MIN(salary) >= 2.0)
                    """
                   
nba_results = pd.read_sql_query(sql_statement, conn)
nba_results.head(50)


,team
0,Chicago Bulls
1,Dallas Mavericks
2,Denver Nuggets
3,Detroit Pistons
4,Golden State Warriors
5,Houston Rockets
6,Los Angeles Lakers
7,Oklahoma City Thunder
8,Philadelphia 76ers
9,Portland Trail Blazers


---
Now, for the first time ever, we will load in a second csv and store it as a second table in the SAME database. 

This second csv/table has some stats on the 20-21 NBA players.

Run the cell below to make this happen.

In [27]:
nba_20_21= pd.read_csv('nba_20-21.csv')   #Open the csv
nba_20_21.to_sql('nba_20_21', conn)   #Dump it into a table called nba_20_21 in the nba_more database

If you run the cell below, you can see that our nba.db file contains two tables, namely, ```nba_15_16``` and ```nba_20_21```.   

In [28]:
#See all the tables included
curs = conn.cursor()
curs.execute("SELECT name FROM sqlite_master WHERE type = 'table'").fetchall()

[('nba_15_16',), ('nba_20_21',)]

### NOTE:  

The ```FROM``` clause in all your select statements must involve one of the above table names.  Our tables are called ```nba_15_16``` and ```nba_20_21```.   

The data from the 20-21 table came from https://basketball.realgm.com/nba/players.  It does not include salary information but it has the following.

* jersey number
* name of player
* position of player
* height
* weight
* yos - Years playing in the NBA
* predraft_team - team they were in before the draft

Let's select everything from the second table, just so that you can get a visual.  Run the cell below.

In [29]:
sql_statement = """SELECT *
                   FROM nba_20_21"""

nba_results = pd.read_sql_query(sql_statement, conn)
nba_results

,index,jersey,name,pos,height,weight,age,team,yos,predraft_team,draft_status,nationality
0,0,15,Precious Achiuwa,SF,81,210,21,Miami Heat,0,Memphis,2020 Rnd 1 Pick 20,Nigeria
1,1,12,Steven Adams,C,84,265,27,New Orleans Pelicans,7,Pittsburgh,2013 Rnd 1 Pick 12,New Zealand
2,2,13,Bam Adebayo,C,73,255,23,Miami Heat,3,Kentucky,2017 Rnd 1 Pick 14,United States
3,3,0,Ty-Shon Alexander,SG,76,195,23,Phoenix Suns,0,Creighton,"2020 NBA Draft, Undrafted",United States
4,4,6,Nickeil Alexander-Walker,SG,77,205,22,New Orleans Pelicans,1,Virginia Tech,2019 Rnd 1 Pick 17,Canada
...,...,...,...,...,...,...,...,...,...,...,...,...
490,490,55,Delon Wright,G,77,185,29,Sacramento Kings,5,Utah,2015 Rnd 1 Pick 20,United States
491,491,21,Thaddeus Young,F,80,235,32,Chicago Bulls,13,Georgia Tech,2007 Rnd 1 Pick 12,United States
492,492,11,Trae Young,PG,73,180,22,Atlanta Hawks,2,Oklahoma,2018 Rnd 1 Pick 5,United States
493,493,40,Cody Zeller,F,84,240,28,Charlotte Hornets,7,Indiana,2013 Rnd 1 Pick 4,United States


----

### Question 7.

Find how many players were on each team in the 2015-2016 season and also show the average salary of each team.  Order the table from the team with the greatest number of players to the least.  Make the column headers match those below.

HINT: You'll get  a 30 by 3 table and the partial answer is shown below.

|                team | num_players | avg_salary |
|--------------------:|------------:|-----------:|
|   Memphis Grizzlies |          21 |   4.466497 |
|   Charlotte Hornets |          18 |   4.672355 |
|  Washington Wizards |          17 |   5.296912 |
|           Utah Jazz |          17 |   3.095993 |
|     Toronto Raptors |          17 |   4.392507 |
|        Phoenix Suns |          17 |   2.971813 |
|                 ... |         ... |        ... |
|  Los Angeles Lakers |          11 |   6.237086 |
|    Dallas Mavericks |          11 |   5.978414 |
|     Detroit Pistons |          10 |   4.221176 |
| Cleveland Cavaliers |          10 |  10.231241 |


In [30]:
sql_statement = """SELECT team, COUNT(player) AS num_players, AVG(salary) AS avg_salary
                   FROM nba_15_16
                   GROUP BY team
                   ORDER BY num_players DESC
                   """

nba_results = pd.read_sql_query(sql_statement, conn)
nba_results


,team,num_players,avg_salary
0,Memphis Grizzlies,21,4.466497
1,Charlotte Hornets,18,4.672355
2,Washington Wizards,17,5.296912
3,Utah Jazz,17,3.095993
4,Toronto Raptors,17,4.392507
5,Phoenix Suns,17,2.971813
6,Oklahoma City Thunder,16,6.052010
7,New Orleans Pelicans,16,5.032163
8,Boston Celtics,15,3.352367
9,Portland Trail Blazers,14,3.246206


----

### Question 8.


For each team 2015-2016, show how many players per position there were and the average salary per position.  The column titles should be shown as follows. Sort the table from the highest average salary to the least.

| position | num_players | avg_salary |
|---------:|------------:|-----------:|

HINT: You'll get  a 5 by 3 table where there are 96 shooting guards who made an average of \$3.988195 million.



In [32]:
sql_statement = """SELECT position, COUNT(player) AS num_players, AVG(salary) AS avg_salary
                   FROM nba_15_16
                   GROUP BY position
                   ORDER BY avg_salary DESC
                   """

nba_results = pd.read_sql_query(sql_statement, conn)
nba_results


,position,num_players,avg_salary
0,C,69,6.082913
1,SF,82,5.532675
2,PG,85,5.165487
3,PF,85,4.951344
4,SG,96,3.988195


Based on your analysis, what position makes the most on avereage?

*Write your answer here*

---

### Question 9.

***Using joins***, show how many players from the 2015-2016 season also played in 2021-21 season.

Hint:  The answer is between 155 and 165.

In [33]:
sql_statement = """SELECT COUNT(player)
                   FROM nba_15_16 JOIN nba_20_21 ON player = name"""

In [34]:
nba_results = pd.read_sql_query(sql_statement, conn)
nba_results

,COUNT(player)
0,158


### Question 10A.

***Using joins***, for each player in the 2015-2016 season, show the player's team in 2015-2016 and the player's team in 2020-2021.  Those players who were not in the NBA in the 2020-2021 season should be included.  Sort by player name and rename the columns so that they look like below.

You'll have 417 rows, since the length fo the 2015-2016 roster is 417.

|          player |          team15_16 |             team20_21 |
|----------------:|-------------------:|----------------------:|
|    Paul Millsap |      Atlanta Hawks |        Denver Nuggets |
|      Al Horford |      Atlanta Hawks | Oklahoma City Thunder |
|  Tiago Splitter |      Atlanta Hawks |                  None |
|     Jeff Teague |      Atlanta Hawks |       Milwaukee Bucks |
|     Kyle Korver |      Atlanta Hawks |                  None |
|             ... |                ... |                   ... |
|       Gary Neal | Washington Wizards |                  None |
|    DeJuan Blair | Washington Wizards |                  None |
| Kelly Oubre Jr. | Washington Wizards |                  None |
|  Garrett Temple | Washington Wizards |         Chicago Bulls |
|    Jarell Eddie | Washington Wizards |                  None |


In [63]:
sql_statement = """SELECT player, nba_15_16.team AS team15_16, nba_20_21.team AS team20_21
                   FROM nba_15_16 LEFT JOIN nba_20_21 ON (nba_15_16.player = nba_20_21.name)
                   """

nba_results = pd.read_sql_query(sql_statement, conn)
nba_results


,player,team15_16,team20_21
0,Paul Millsap,Atlanta Hawks,Denver Nuggets
1,Al Horford,Atlanta Hawks,Oklahoma City Thunder
2,Tiago Splitter,Atlanta Hawks,None
3,Jeff Teague,Atlanta Hawks,Milwaukee Bucks
4,Kyle Korver,Atlanta Hawks,None
...,...,...,...
412,Gary Neal,Washington Wizards,None
413,DeJuan Blair,Washington Wizards,None
414,Kelly Oubre Jr.,Washington Wizards,None
415,Garrett Temple,Washington Wizards,Chicago Bulls


### Question 10B.

Modify the query in 7A sp that the resulting table shows only those players who are not on the 2020-2021 roster.  ***Use the NULL keyword to do this.***

HINT:  There are 417 folks on the 2015-2016 roster and you know how many of them played in the 20-21 season from a previous question.  So how many rows should be in this table?

|      player     |      team15_16     | team20_21 |
|:---------------:|:------------------:|:---------:|
|  Tiago Splitter |    Atlanta Hawks   |    None   |
|   Kyle Korver   |    Atlanta Hawks   |    None   |
|       ...       |         ...        |    ...    |
| Kelly Oubre Jr. | Washington Wizards |    None   |
|   Jarell Eddie  | Washington Wizards |    None   |


In [67]:
sql_statement = """SELECT player, nba_15_16.team AS team15_16, nba_20_21.team AS team20_21
                   FROM nba_15_16 LEFT JOIN nba_20_21 ON (nba_15_16.player = nba_20_21.name)
                   WHERE nba_20_21.team IS NULL
                   """

nba_results = pd.read_sql_query(sql_statement, conn)
nba_results


,player,team15_16,team20_21
0,Tiago Splitter,Atlanta Hawks,None
1,Kyle Korver,Atlanta Hawks,None
2,Thabo Sefolosha,Atlanta Hawks,None
3,Walter Tavares,Atlanta Hawks,None
4,Jason Richardson,Atlanta Hawks,None
...,...,...,...
254,Ramon Sessions,Washington Wizards,None
255,Gary Neal,Washington Wizards,None
256,DeJuan Blair,Washington Wizards,None
257,Kelly Oubre Jr.,Washington Wizards,None


### Question 11.

Using joins, show any players from the 2015-2016 season who are still playing but on a different team in 20-21. Be sure to show the player and both teams, renaming your columns as below.

The answer is shown below.

|     |      player     |      team15_16     |       team20_21       |
|:---:|:---------------:|:------------------:|:---------------------:|
|  0  |   Paul Millsap  |    Atlanta Hawks   |     Denver Nuggets    |
|  1  |    Al Horford   |    Atlanta Hawks   | Oklahoma City Thunder |
|  2  |   Jeff Teague   |    Atlanta Hawks   |    Milwaukee Bucks    |
|  3  |    Mike Scott   |    Atlanta Hawks   |  Philadelphia Sixers  |
|  4  |  Kent Bazemore  |    Atlanta Hawks   | Golden State Warriors |
| ... |       ...       |         ...        |          ...          |
| 129 |   Rodney Hood   |      Utah Jazz     |    Toronto Raptors    |
| 130 |    John Wall    | Washington Wizards |    Houston Rockets    |
| 131 | Markieff Morris | Washington Wizards |   Los Angeles Lakers  |
| 132 |   Jared Dudley  | Washington Wizards |   Los Angeles Lakers  |
| 133 |  Garrett Temple | Washington Wizards |     Chicago Bulls     |

In [69]:
sql_statement = """SELECT player, nba_15_16.team AS team15_16, nba_20_21.team AS team20_21
                   FROM nba_15_16 LEFT JOIN nba_20_21 ON (nba_15_16.player = nba_20_21.name)
                   WHERE nba_15_16.team != nba_20_21.team
                   """

nba_results = pd.read_sql_query(sql_statement, conn)
nba_results


,player,team15_16,team20_21
0,Paul Millsap,Atlanta Hawks,Denver Nuggets
1,Al Horford,Atlanta Hawks,Oklahoma City Thunder
2,Jeff Teague,Atlanta Hawks,Milwaukee Bucks
3,Mike Scott,Atlanta Hawks,Philadelphia Sixers
4,Kent Bazemore,Atlanta Hawks,Golden State Warriors
...,...,...,...
129,Rodney Hood,Utah Jazz,Toronto Raptors
130,John Wall,Washington Wizards,Houston Rockets
131,Markieff Morris,Washington Wizards,Los Angeles Lakers
132,Jared Dudley,Washington Wizards,Los Angeles Lakers


### Question 12.

Using joins, ***for each team***, show how many players from the 2015-2016 season also played in 2021-21 season.  Also show the average age and  average 15-16 salary of these players.  Sort by team name ascending and rename the columns so that they look like below.

You'll get 30 rows for the 30 teams.

|                team | num_in_both |   avg_age | avg_salary |
|--------------------:|------------:|----------:|-----------:|
|       Atlanta Hawks |           7 | 31.571429 |   6.724702 |
|      Boston Celtics |           5 | 28.800000 |   4.389403 |
|       Brooklyn Nets |           4 | 30.750000 |   4.374236 |
|   Charlotte Hornets |           4 | 29.500000 |   8.090966 |
|       Chicago Bulls |           6 | 29.833333 |   7.137244 |
| Cleveland Cavaliers |           4 | 31.750000 |   18.33196 |
| ...                 | ...         | ...       | ...        |


In [76]:
sql_statement = """SELECT nba_15_16.team, COUNT(name) AS num_in_both, AVG(nba_20_21.age) AS avg_age, AVG(nba_15_16.salary) AS avg_salary
                   FROM nba_15_16 LEFT JOIN nba_20_21 ON (nba_15_16.player = nba_20_21.name)
                   WHERE nba_15_16.player = nba_20_21.name
                   GROUP BY nba_15_16.team
                   """

nba_results = pd.read_sql_query(sql_statement, conn)
nba_results


,team,num_in_both,avg_age,avg_salary
0,Atlanta Hawks,7,31.571429,6.724702
1,Boston Celtics,5,28.800000,4.389403
2,Brooklyn Nets,4,30.750000,4.374236
3,Charlotte Hornets,4,29.500000,8.090966
4,Chicago Bulls,6,29.833333,7.137244
5,Cleveland Cavaliers,4,31.750000,18.331968
6,Dallas Mavericks,1,33.000000,1.270964
7,Denver Nuggets,5,28.000000,4.451963
8,Detroit Pistons,6,29.000000,5.111883
9,Golden State Warriors,6,30.833333,9.641412


### Question 13

***Using the "RIGHT" kind of join,*** :) show the name of the players in the 20-21 season, their current team, and the 2015-2016 team. 

If they dont' have a 2015-2016 team, show the phrase "NEW".

The answer is shown.

|     |           name           |       team20_21      |       team15_16       |
|:---:|:------------------------:|:--------------------:|:---------------------:|
|  0  |     Precious Achiuwa     |      Miami Heat      |          NEW          |
|  1  |       Steven Adams       | New Orleans Pelicans | Oklahoma City Thunder |
|  2  |        Bam Adebayo       |      Miami Heat      |          NEW          |
|  3  |     Ty-Shon Alexander    |     Phoenix Suns     |          NEW          |
|  4  | Nickeil Alexander-Walker | New Orleans Pelicans |          NEW          |
| ... |            ...           |          ...         |          ...          |
| 490 |       Delon Wright       |   Sacramento Kings   |    Toronto Raptors    |
| 491 |      Thaddeus Young      |     Chicago Bulls    |     Brooklyn Nets     |
| 492 |        Trae Young        |     Atlanta Hawks    |          NEW          |
| 493 |        Cody Zeller       |   Charlotte Hornets  |   Charlotte Hornets   |
| 494 |        Ivica Zubac       | Los Angeles Clippers |          NEW          |

In [87]:
sql_statement = """SELECT name, nba_20_21.team AS team20_21, COALESCE(nba_15_16.team,'NEW') AS team15_16
                   FROM nba_20_21 LEFT JOIN nba_15_16 ON (nba_20_21.name = nba_15_16.player)
                   
                   """

nba_results = pd.read_sql_query(sql_statement, conn)
nba_results


,name,team20_21,team15_16
0,Precious Achiuwa,Miami Heat,NEW
1,Steven Adams,New Orleans Pelicans,Oklahoma City Thunder
2,Bam Adebayo,Miami Heat,NEW
3,Ty-Shon Alexander,Phoenix Suns,NEW
4,Nickeil Alexander-Walker,New Orleans Pelicans,NEW
...,...,...,...
490,Delon Wright,Sacramento Kings,Toronto Raptors
491,Thaddeus Young,Chicago Bulls,Brooklyn Nets
492,Trae Young,Atlanta Hawks,NEW
493,Cody Zeller,Charlotte Hornets,Charlotte Hornets


----

### Question 14.

For the players in 20-21, show the average weight and the min/max/average height per position.  Sort the table by the average height from high to low.  Rename your columns to have nicely formatted headers that explain what data is in the column.

There should be 9 rows.

In [98]:
sql_statement = """SELECT pos AS Position, AVG(weight) AS Average_Weight, MIN(Height) AS Minimum_Height, MAX(Height) AS Maximum_Height, AVG(Height) AS Average_Height
                   FROM nba_20_21
                   GROUP BY pos
                   ORDER BY AVG(Height) DESC"""

nba_results = pd.read_sql_query(sql_statement, conn)
nba_results


,Position,Average_Weight,Minimum_Height,Maximum_Height,Average_Height
0,C,250.769231,73,89,79.596154
1,F,223.911765,73,85,79.323529
2,GF,209.769231,75,81,78.307692
3,SF,218.835616,73,84,78.232877
4,PF,233.909091,73,84,77.000000
5,FC,248.428571,73,87,77.000000
6,SG,202.772152,61,81,76.810127
7,G,197.811594,72,81,75.840580
8,PG,190.344262,61,79,73.721311


### Question 15.

Does it appear that there is a significant difference between the height and weight of any of the positions?  Explain.

So this table goes really specific whhen it comes to players who play multiple positions. The players who play guard positions are traditionally smaller and faster. Players who play PF and Centers are usally much taller than everyone else because they need to guard the paint near the rim. So yes, guards and big men are significantly different in size.

----
Let's close the datbase connection to the nba.

In [99]:
conn.close()

---
You're done your last HW!  Thank you for you a great semester!